In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
matpower = pd.read_csv("./7KbusGen.csv")
eia = pd.read_csv("./EIA_860_generator_list.csv")
# split eia plant code_gen ID into plant code and gen ID
matpower["Plant Code"] = matpower["EIA plant code_gen ID"].str.split("_", expand=True)[0].astype(int)
matpower["Generator ID"] = matpower["EIA plant code_gen ID"].str.split("_", expand=True)[1].astype(str)
matpower.drop("EIA plant code_gen ID", axis=1, inplace=True)
matpower.columns


Index(['Number of Bus', 'Name of Bus', 'ID',
       'Start-Up Costs ($/MW of capacity/start)',
       'Shut-Down Costs ($/MW/shutdown)', 'Cold Start-Up Time (hours)',
       'Warm Start-Up Time (hours)', 'Hot Start-Up Time (hours)',
       'Minimum Up Time (hours)', 'Minimum Down Time (hours)',
       'Ramp Rate Up and Down (%/min) ', ' Variable O&M Costs ($/MWh)',
       'Fuel Costs ($/MMBtu)', 'Sub Num of Bus', 'Status', 'Gen MW',
       'Gen Mvar', 'Min MW', 'Max MW', 'AGC', '# Cost Curve Points', 'IOB',
       'IOC', 'Variable O&M', 'Fuel Cost', 'IOD', 'AVR', 'RegBus Num',
       'Fuel Type', 'Set Volt', 'Min Mvar', 'Max Mvar', 'Enforce MW Limits',
       'Part. Factor', 'Cost Model', 'Plant Code', 'Generator ID'],
      dtype='object')

In [3]:
eia.columns

Index(['Utility ID', 'Utility Name', 'Plant Code', 'Plant Name', 'State',
       'County', 'Generator ID', 'Technology', 'Prime Mover', 'Unit Code',
       'Ownership', 'Duct Burners',
       'Can Bypass Heat Recovery Steam Generator?',
       'RTO/ISO LMP Node Designation',
       'RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC',
       'Nameplate Capacity (MW)', 'Nameplate Power Factor',
       'Summer Capacity (MW)', 'Winter Capacity (MW)', 'Minimum Load (MW)',
       'Uprate or Derate Completed During Year',
       'Month Uprate or Derate Completed', 'Year Uprate or Derate Completed',
       'Status', 'Synchronized to Transmission Grid', 'Operating Month',
       'Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
       'Associated with Combined Heat and Power System', 'Sector Name',
       'Sector', 'Topping or Bottoming', 'Energy Source 1', 'Energy Source 2',
       'Energy Source 3', 'Energy Source 4', 'Energy Source 5',
       'Ene

In [6]:
eia["Energy Source 1"].unique()

array(['NG', 'SUB', 'WAT', 'DFO', 'NUC', 'RC', 'LIG', 'OG', 'PC', 'BLQ',
       'WH', 'BFG', 'WDS', 'PUR', 'LFG', 'AB', 'WND', 'MWH', 'SUN'],
      dtype=object)

In [10]:
cross = pd.merge(matpower[["Plant Code", "Generator ID", "Fuel Type"]], eia[["Plant Code", "Generator ID", "Energy Source 1"]], how="left", on=["Plant Code", "Generator ID"])
cross

,Plant Code,Generator ID,Fuel Type,Energy Source 1
0,3460,1,NG (Natural Gas),NG
1,3460,2,NG (Natural Gas),NG
2,3464,73,NG (Natural Gas),NG
3,3464,74,NG (Natural Gas),NG
4,3464,81,NG (Natural Gas),NG
...,...,...,...,...
726,55153,STG2,NG (Natural Gas),NG
727,55153,CTG1,NG (Natural Gas),NG
728,55153,CTG2,NG (Natural Gas),NG
729,55153,CTG3,NG (Natural Gas),NG


In [8]:
# Fuel Conversions
dic = {"Coal":["ANT", "BIT", "LIG", "SUB", "WC", "RC"], 
    "Petroleum": ["DFO", "JF", "KER", "PC", "RFO", "WO"],
    "Natural Gas": ["BFG", "NG", "OG", "PG", "SG", "SGC"],
    "Renewable": ["AB", "NSW", "OBS", "WDS", "OBL", "SLW", "BLQ", "WDL", "LFG", "OBG"],
    "Solar": ["SUN"],
    "Wind": ["WND"],
    "Nuclear": ["NUC"],
    "Hydro": ["WAT"],
    "Geothermal": ["GEO"],
    "Other": ["MWH", "WH", "PUR"]
    }


#iterates through data and changed value of Energy Source 1 to the key value in the dic
for index, row in cross.iterrows():
    for key, value in dic.items():
        if row["Energy Source 1"] in value:
            cross.at[index, "Energy Source 1"] = key